# Punto 1 – Modelado Predictivo de Ventas (Retail)

**Objetivo:**
- Construir 3 modelos *baseline* (Regresión Lineal, Random Forest, Gradient Boosting) con validación cruzada (k=5) y métricas **R²** y **MAPE**.
- Seleccionar el mejor y luego **optimizar Gradient Boosting** (XGBoost si está disponible; si no, `GradientBoostingRegressor`).

**Entradas esperadas:**
- `/mnt/data/Punto1.tiendas_100.csv` (train con `ventas_m24`).
- `/mnt/data/Punto1.tiendas_10_no_target.csv` (10 nuevas tiendas sin target).

**Salidas:**
- Tabla comparativa de resultados baseline (CV=5).
- Mejor modelo tras *random search* y su tabla de **importancia de variables**.
- **Predicciones** para las 10 tiendas nuevas (archivo CSV listo para el Google Sheet del parcial).


In [3]:
import os, warnings
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd
from sklearn.model_selection import KFold, cross_validate, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Intentar XGBoost; si no existe, se usará GradientBoostingRegressor
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

DATA_TRAIN = '../data/clean/Punto1.tiendas_100_clean.csv'
DATA_NEW   = '../data/clean/Punto1.tiendas_10_no_target_clean.csv'
TARGET = 'ventas_m24'

print('XGBoost disponible?', HAS_XGB)
print('Archivos presentes en /mnt/data:', os.listdir('/mnt/data') if os.path.exists('/mnt/data') else 'No existe /mnt/data')

XGBoost disponible? False
Archivos presentes en /mnt/data: No existe /mnt/data


## 1) Carga de datos y definición de *features*
Detectamos automáticamente columnas numéricas y categóricas (por tipo).

In [4]:
assert os.path.exists(DATA_TRAIN), 'No se encontró el archivo de entrenamiento: ' + DATA_TRAIN
assert os.path.exists(DATA_NEW),   'No se encontró el archivo de nuevas tiendas: ' + DATA_NEW

df_train = pd.read_csv(DATA_TRAIN)
df_new   = pd.read_csv(DATA_NEW)

assert TARGET in df_train.columns, f'La columna target {TARGET} no está en el train'

cat_cols = [c for c in df_train.columns if df_train[c].dtype == 'object' and c != TARGET]
num_cols = [c for c in df_train.columns if c not in cat_cols + [TARGET]]

print('Shape train:', df_train.shape)
print('Shape nuevas tiendas:', df_new.shape)
print('Numéricas:', num_cols)
print('Categóricas:', cat_cols)

X = df_train.drop(columns=[TARGET])
y = df_train[TARGET].values

Shape train: (100, 27)
Shape nuevas tiendas: (10, 25)
Numéricas: ['lat', 'lon', 'pop_100m', 'pop_300m', 'pop_500m', 'commerces', 'gas_stations', 'malls', 'foot_traffic', 'car_traffic', 'socio_level', 'viviendas_100m', 'oficinas_100m', 'viviendas_pobreza', 'competencia', 'tiendas_peq', 'pop_total', 'pop_comp_100m', 'pop_comp_300m', 'pop_comp_500m', 'pop_comp_total', 'oficinas_comp', 'viviendas_comp', 'is_outlier']
Categóricas: ['Tienda', 'store_cat']


## 2) Métricas y *preprocessing*
- **MAPE** como métrica principal (evitando división por cero).
- Dos *preprocessors*: uno con *scaling* para modelo lineal; otro sin *scaling* para árboles.

In [5]:
def mape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, eps, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0

mape_scorer = make_scorer(mape, greater_is_better=False)

preprocess_linear = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ]), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ]), cat_cols),
])

preprocess_tree = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ]), cat_cols),
])

cv = KFold(n_splits=5, shuffle=True, random_state=42)

## 3) Modelos *baseline* (Regresión Lineal, Random Forest, Gradient Boosting)
Calculamos **R²** y **MAPE** en CV=5 para cada modelo.

In [6]:
results = []

# A) Regresión Lineal con log-transform del target
lin = Pipeline([
    ('pre', preprocess_linear),
    ('reg', LinearRegression()),
])
lin_ttr = TransformedTargetRegressor(regressor=lin, func=np.log1p, inverse_func=np.expm1)
scores_lin = cross_validate(lin_ttr, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
    'model': 'LinearRegression (log-target)',
    'R2_mean': np.mean(scores_lin['test_R2']),
    'R2_std':  np.std(scores_lin['test_R2']),
    'MAPE_mean': -np.mean(scores_lin['test_MAPE']),
    'MAPE_std':  np.std(scores_lin['test_MAPE']),
})

# B) Random Forest baseline
rf = Pipeline([
    ('pre', preprocess_tree),
    ('rf', RandomForestRegressor(n_estimators=300, min_samples_leaf=3, random_state=42, n_jobs=-1)),
])
scores_rf = cross_validate(rf, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
    'model': 'RandomForestRegressor (baseline)',
    'R2_mean': np.mean(scores_rf['test_R2']),
    'R2_std':  np.std(scores_rf['test_R2']),
    'MAPE_mean': -np.mean(scores_rf['test_MAPE']),
    'MAPE_std':  np.std(scores_rf['test_MAPE']),
})

# C) Gradient Boosting baseline (XGB si está disponible; si no, sklearn GBR)
if HAS_XGB:
    gb_est = XGBRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=4,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
        random_state=42, n_jobs=-1, tree_method='hist'
    )
    gb_name = 'XGBRegressor (baseline)'
else:
    gb_est = GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=3,
        subsample=0.9, random_state=42
    )
    gb_name = 'GradientBoostingRegressor (baseline)'

gb = Pipeline([
    ('pre', preprocess_tree),
    ('gb', gb_est),
])
scores_gb = cross_validate(gb, X, y, cv=cv, scoring={'R2': 'r2', 'MAPE': mape_scorer}, n_jobs=-1)
results.append({
    'model': gb_name,
    'R2_mean': np.mean(scores_gb['test_R2']),
    'R2_std':  np.std(scores_gb['test_R2']),
    'MAPE_mean': -np.mean(scores_gb['test_MAPE']),
    'MAPE_std':  np.std(scores_gb['test_MAPE']),
})

df_results_baseline = pd.DataFrame(results).sort_values('MAPE_mean')
df_results_baseline

,model,R2_mean,R2_std,MAPE_mean,MAPE_std
1,RandomForestRegressor (baseline),0.827256,0.084777,16.086279,3.486338
2,GradientBoostingRegressor (baseline),0.857887,0.087602,16.250015,3.388854
0,LinearRegression (log-target),-1.400612,3.336512,27.566383,5.103824


## 4) Optimización de Gradient Boosting (*RandomizedSearchCV*)
Se optimizan hiperparámetros clave para mejorar MAPE y controlar sobreajuste.

In [7]:
if HAS_XGB:
    gb_search = Pipeline([
        ('pre', preprocess_tree),
        ('gb', XGBRegressor(random_state=42, n_jobs=-1, tree_method='hist')),
    ])
    param_dist = {
        'gb__n_estimators': [200, 400, 600, 800, 1000],
        'gb__max_depth': [2, 3, 4, 5, 6],
        'gb__learning_rate': np.linspace(0.01, 0.2, 10),
        'gb__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__reg_lambda': [0.0, 0.1, 0.5, 1.0, 5.0, 10.0],
    }
else:
    gb_search = Pipeline([
        ('pre', preprocess_tree),
        ('gb', GradientBoostingRegressor(random_state=42)),
    ])
    param_dist = {
        'gb__n_estimators': [200, 400, 600, 800, 1000],
        'gb__max_depth': [2, 3, 4, 5],
        'gb__learning_rate': np.linspace(0.01, 0.2, 10),
        'gb__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gb__max_features': [None, 'sqrt', 0.6, 0.8],
        'gb__min_samples_leaf': [1, 2, 3, 5, 10],
    }

rand_search = RandomizedSearchCV(
    estimator=gb_search,
    param_distributions=param_dist,
    n_iter=40, cv=cv,
    scoring={'R2': 'r2', 'MAPE': mape_scorer},
    refit='MAPE', random_state=42, n_jobs=-1, verbose=1
)
rand_search.fit(X, y)
best_model = rand_search.best_estimator_
best_mape = -rand_search.best_score_
best_params = rand_search.best_params_

print('Mejor MAPE (CV):', round(best_mape, 3))
print('Mejores parámetros:', best_params)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


KeyboardInterrupt: 

## 5) Importancia de variables del mejor modelo
Se extraen importancias del modelo (árboles) o coeficientes (si fuera lineal).

In [ ]:
# Nombres de columnas transformadas (num + dummies de cat)
pre = best_model.named_steps['pre']
try:
    num_out = np.array([c for c in num_cols])
    ohe = pre.named_transformers_['cat'].named_steps['onehot']
    cat_out = ohe.get_feature_names_out(cat_cols)
    feature_names = np.concatenate([num_out, cat_out])
except Exception:
    feature_names = np.array(num_cols + cat_cols)

importances = None
try:
    importances = best_model.named_steps['gb'].feature_importances_
except Exception:
    try:
        importances = np.abs(best_model.named_steps['reg'].coef_)
    except Exception:
        importances = np.zeros(len(feature_names))

feat_imp = pd.DataFrame({'feature': feature_names[:len(importances)], 'importance': importances})\
            .sort_values('importance', ascending=False).reset_index(drop=True)
feat_imp.head(25)

## 6) Predicciones para las 10 nuevas tiendas
Se aplica el pipeline completo `best_model` sobre `df_new` para obtener `ventas_m24_pred`.

In [ ]:
preds = best_model.predict(df_new)
df_preds = df_new.copy()
df_preds['ventas_m24_pred'] = preds
df_preds.head(10)

## 7) Guardado de resultados para el entregable
Se exporta:
- `resultados_baseline_punto1.csv`
- `feature_importances_punto1.csv`
- `predicciones_punto1.csv` (10 tiendas)

In [ ]:
out_baseline = '/mnt/data/resultados_baseline_punto1.csv'
out_featimp = '/mnt/data/feature_importances_punto1.csv'
out_preds   = '/mnt/data/predicciones_punto1.csv'

df_results_baseline.to_csv(out_baseline, index=False)
feat_imp.to_csv(out_featimp, index=False)
df_preds.to_csv(out_preds, index=False)

print('Archivos guardados:')
print(' -', out_baseline)
print(' -', out_featimp)
print(' -', out_preds)

### Notas finales
- **Selección de modelo**: elegir el que minimice **MAPE** (promedio CV). Reportar también **R²**.
- **Interpretabilidad**: acompañar el modelo final con ranking de importancia y una breve explicación de los *drivers* (tráfico, competencia, población, etc.).
- **Entrega (Google Sheets)**: pegar `predicciones_punto1.csv` y el top de `feature_importances_punto1.csv` en la hoja "Punto 1", junto con un resumen del modelo y métricas.